# Final Model

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader, Dataset
import json
from tqdm import tqdm

In [10]:
# Check if GPU is available and move the model accordingly
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


## Loading Models and Tokenizers

In [58]:
# Initialize the tokenizer and model
qna_tokenizer = AutoTokenizer.from_pretrained('../3)QnA/model_params_qna')
qna_model = AutoModelForQuestionAnswering.from_pretrained('../3)QnA/model_params_qna').to(device)

In [59]:
# Initialize the tokenizer and model
translation_tokenizer = AutoTokenizer.from_pretrained('../2)Translation/model_params_translate')
translation_model = AutoModelForSeq2SeqLM.from_pretrained('../2)Translation/model_params_translate').to(device)

## Part 1: QnA

In [67]:
# make answers predictions
def get_qna_preds(context, question):
    with torch.no_grad():
        inputs = qna_tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
        outputs = qna_model(**inputs)
        answer_start = torch.argmax(outputs[0])
        answer_end = torch.argmax(outputs[1]) + 1
        answer = qna_tokenizer.convert_tokens_to_string(qna_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

## Part 2: Translation

In [89]:
def get_translation_preds(text_data):
    with torch.no_grad():
        inputs = translation_tokenizer(text_data, return_tensors='pt').to(device)
        outputs = translation_model.generate(**inputs)
        answer = translation_tokenizer.batch_decode(outputs)
        final_ans = answer[0][6:-4] # omitting some tokens
    return final_ans

## Final Function

In [90]:
def display_results(context, questions, eng_answer, fren_answer):
    print(f'Question: {questions}')
    print(f'English Prediction: {eng_answer}')
    print(f'Final French Prediction: {fren_answer}\n')

In [91]:
def get_translated_qna(context, questions):
    for question in questions:
        # Get the answer from qna_model
        eng_answer = get_qna_preds(context, question)
        # Translate answer
        fren_answer = get_translation_preds(eng_answer)
        # Display answers
        display_results(context, question, eng_answer, fren_answer)

## Results

In [92]:
context = """Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer,
          songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing
          and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child.
          Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time.
          Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide,
          earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""


questions = ["For whom the passage is talking about?",
             "When did Beyonce born?",
             "Where did Beyonce born?",
             "What is Beyonce's nationality?",
             "Who was the Destiny's group manager?",
             "What name has the Beyoncé's debut album?",
             "How many Grammy Awards did Beyonce earn?",
             "When did the Beyoncé's debut album release?",
             "Who was the lead singer of R&B girl-group Destiny's Child?"]

In [93]:
get_translated_qna(context, questions)

Question: For whom the passage is talking about?
English Prediction: Beyoncé
Final French Prediction: Beyoncé

Question: When did Beyonce born?
English Prediction:  September 4, 1981
Final French Prediction: le 4 septembre 1981

Question: Where did Beyonce born?
English Prediction:  Houston, Texas
Final French Prediction: Houston (Texas)

Question: What is Beyonce's nationality?
English Prediction:  American
Final French Prediction: Américain

Question: Who was the Destiny's group manager?
English Prediction:  Mathew Knowles
Final French Prediction: Mathew Knowles

Question: What name has the Beyoncé's debut album?
English Prediction:  Dangerously in Love
Final French Prediction: Dangereusement amoureux

Question: How many Grammy Awards did Beyonce earn?
English Prediction:  five
Final French Prediction: cinq

Question: When did the Beyoncé's debut album release?
English Prediction: 2003
Final French Prediction: en 2003

Question: Who was the lead singer of R&B girl-group Destiny's Chi